In [1]:
# from rbbm_src.labelling_func_src.src.utils import lf_constraint_solve
from rbbm_src.labelling_func_src.src.lfs_tree import keyword_labelling_func_builder
from rbbm_src.labelling_func_src.src.TreeRules import SPAM, HAM, ABSTAIN, PredicateNode
from rbbm_src.labelling_func_src.src.LFRepair import run_snorkel, populate_violations
from rbbm_src.labelling_func_src.src.classes import clean_text

import re
import psycopg2
import pandas as pd
from snorkel.labeling import (
	LabelingFunction, 
	labeling_function, 
	PandasLFApplier, 
	LFAnalysis,
	filter_unlabeled_dataframe
	)
from snorkel.labeling.model import MajorityLabelVoter, LabelModel
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import matplotlib.patches as mpatches
import pulp
from sklearn.metrics import accuracy_score, classification_report

from rbbm_src.labelling_func_src.src.KeyWordRuleMiner import KeyWordRuleMiner 
# sample user confirmation and complaints
import random
from collections import deque
import numpy as np
import pickle

[nltk_data] Downloading package words to /home/opc/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /home/opc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2024-10-13 01:23:52.490036: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-13 01:23:52.542562: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-13 01:23:52.543910: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
DEBUG 2024-10-13 01:23:53,379 [tpu_cluster_resolver.py:<module>:32] Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install c

In [2]:
from rbbm_src.labelling_func_src.src.example_tree_rules import gen_amazon_funcs

In [3]:
def calculate_coverage(df, lf_column):
    total_instances = len(df)
    labeled_instances = df[lf_column] != -1
    coverage = labeled_instances.sum() / total_instances
    return coverage

def calculate_accuracy(df, lf_column, ground_truth_column):
    labeled_instances = df[df[lf_column] != -1]
    correct_labels = (labeled_instances[lf_column] == labeled_instances[ground_truth_column]).sum()
    total_labeled = len(labeled_instances)
    if total_labeled == 0:
        return 0.0  # Avoid division by zero if no instances are labeled
    accuracy = correct_labels / total_labeled
    return accuracy

In [4]:
kwords_and_label_re = re.compile('\((.*)\)->(\w+)')
mapping_dict = {'negative':SPAM, 'positive':HAM}

rules_written = ['(love,terrific,amazing)->positive',
                 '(great,helpful,five stars)->positive',
                 '(repeat customer)->positive',
                 '(beautiful, excellent)->positive',
                 '(favorite,nice)->positive',
                 '(comfortable)->positive',
                 '(one star, terrible)->negative',
                 '(broken, poor)->negative',
                 '(cheap, only)->negative',
                 '(crap,big,small)->negative',
                 '(difficult)->negative',
                 '(not, depress)->negative',
                 '(boring)->negative'
                 ]
manual_tree_rules = []

for r in rules_written:
    try:
        kword_string, label_text = kwords_and_label_re.search(r).groups()
        kwords = [x.strip() for x in kword_string.split(',')]
        label = mapping_dict[label_text.strip()]
        manual_tree_rules.append(keyword_labelling_func_builder(keywords=kwords, expected_label=label))
    except:
        print(f'not captured properly for rule {r}')
manual_funcs = [f.gen_label_rule() for f in manual_tree_rules]


In [5]:
conn = psycopg2.connect(dbname='label', user='postgres')

In [6]:
sentences_df=pd.read_sql(f'SELECT * FROM amazon01', conn)
sentences_df = sentences_df.rename(columns={"class": "expected_label", "content": "old_text"})
sentences_df['text'] = sentences_df['old_text'].apply(lambda s: clean_text(s))
sentences_df = sentences_df[~sentences_df['text'].isna()]

/tmp/ipykernel_3802/3647735799.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sentences_df=pd.read_sql(f'SELECT * FROM amazon01', conn)


In [7]:
from collections import defaultdict
individual_func_stats = defaultdict(dict)
amazon_treerules = gen_amazon_funcs()
amazon_funcs = [f.gen_label_rule() for f in amazon_treerules]

In [8]:
# for f in amazon_funcs:
#     print(f)
#     sentences_df['lf_labeled'] = sentences_df.apply(f, axis=1)
#     cov=calculate_coverage(df=sentences_df, lf_column='lf_labeled')
#     acc=calculate_accuracy(df=sentences_df, lf_column='lf_labeled', ground_truth_column='expected_label')
#     individual_func_stats[f]={'cov':cov, 'acc':acc}
#     print(f"cov:{round(cov,2)}, acc:{round(acc,2)}")
#     print('\n')

In [9]:

# for f in manual_funcs:
#     print(f)
#     sentences_df['lf_labeled'] = sentences_df.apply(f, axis=1)
#     cov=calculate_coverage(df=sentences_df, lf_column='lf_labeled')
#     acc=calculate_accuracy(df=sentences_df, lf_column='lf_labeled', ground_truth_column='expected_label')
#     individual_func_stats[f]={'cov':cov, 'acc':acc}
#     print(f"cov:{round(cov,4)}, acc:{round(acc,4)}")
#     print('---------------------------------------\n')

In [10]:
applier = PandasLFApplier(lfs=amazon_funcs)
initial_vectors = applier.apply(df=sentences_df, progress_bar=False)
model = LabelModel(cardinality=2, verbose=True, device='cpu')
model.fit(L_train=initial_vectors, n_epochs=500, log_freq=100, seed=123)

probs_test= model.predict_proba(L=initial_vectors)

INFO 2024-10-13 01:24:11,752 [label_model.py:fit:905] Computing O...
INFO 2024-10-13 01:24:11,760 [label_model.py:fit:911] Estimating \mu...
 58%|██████████████████████████████▏                     | 290/500 [00:00<00:00, 857.23epoch/s]INFO 2024-10-13 01:24:12,178 [logger.py:log:79] [300 epochs]: TRAIN:[loss=0.033]
INFO 2024-10-13 01:24:12,267 [logger.py:log:79] [400 epochs]: TRAIN:[loss=0.033]
100%|████████████████████████████████████████████████████| 500/500 [00:00<00:00, 844.32epoch/s]
INFO 2024-10-13 01:24:12,358 [label_model.py:fit:977] Finished Training


In [11]:
df_sentences_filtered, probs_test_filtered, filtered_vectors, df_no_signal  = filter_unlabeled_dataframe(
        X=sentences_df, y=probs_test, L=initial_vectors
)	

df_sentences_filtered = df_sentences_filtered.reset_index(drop=True)
prob_diffs = [abs(t[0]-t[1]) for t in probs_test_filtered]
prob_diffs_tuples = [(t[0],t[1]) for t in probs_test_filtered]
df_sentences_filtered['model_pred_diff'] = pd.Series(prob_diffs)
df_sentences_filtered['model_pred_prob_tuple'] = pd.Series(prob_diffs_tuples)
df_sentences_filtered['model_pred'] = pd.Series(model.predict(L=filtered_vectors))

wrong_preds = df_sentences_filtered[(df_sentences_filtered['expected_label']!=df_sentences_filtered['model_pred'])]
# df_sentences_filtered.to_csv('predictions_shakira.csv', index=False)
# logger.critical(wrong_preds)
accuracy=(len(df_sentences_filtered)-len(wrong_preds))/len(df_sentences_filtered)

print(f"""
    out of {len(sentences_df)} sentences, {len(df_sentences_filtered)} actually got at least one signal to \n
    make prediction. Out of all the valid predictions, we have {len(wrong_preds)} wrong predictions, \n
    accuracy = {(len(df_sentences_filtered)-len(wrong_preds))/len(df_sentences_filtered)} 
    """)


    out of 19946 sentences, 18915 actually got at least one signal to 

    make prediction. Out of all the valid predictions, we have 6918 wrong predictions, 

    accuracy = 0.6342585249801744 
    


In [12]:
lf_vectors = filtered_vectors
ground_truth = df_sentences_filtered['expected_label']
snorkel_predictions = df_sentences_filtered['model_pred']
snorkel_probs = df_sentences_filtered['model_pred_diff']

In [13]:
df_sentences_filtered['vectors'] = pd.Series([",".join(map(str, t)) for t in filtered_vectors])

In [14]:
correct_predictions = (snorkel_predictions == ground_truth)
incorrect_predictions = (snorkel_predictions != ground_truth)

In [15]:
filtered_vectors[correct_predictions]

array([[-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1,  1],
       [ 0, -1, -1, ..., -1, -1, -1],
       ...,
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1,  0, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1]])

In [16]:
correct_preds_by_snorkel = df_sentences_filtered[correct_predictions].reset_index(drop=True)
wrong_preds_by_snorkel = df_sentences_filtered[incorrect_predictions].reset_index(drop=True)

user_confirm_df = correct_preds_by_snorkel.sample(n=10, random_state=42)
user_complaints_df = wrong_preds_by_snorkel.sample(n=10, random_state=42)

random_confirm_indices = user_confirm_df.index
random_complaints_indices = user_complaints_df.index
random_user_confirms_vecs = filtered_vectors[correct_predictions][random_confirm_indices]
random_user_complaints_vecs = filtered_vectors[incorrect_predictions][random_complaints_indices]

user_input_df = pd.concat([user_confirm_df, user_complaints_df])

gts = user_input_df['expected_label'].reset_index(drop=True)

manual_rule_applier = PandasLFApplier(lfs=manual_funcs)
vectors_manual = manual_rule_applier.apply(df=user_input_df, progress_bar=False)
user_vecs = np.vstack((random_user_confirms_vecs, random_user_complaints_vecs))

df_vectors_manual = pd.DataFrame(vectors_manual, columns=[f'nlf_{i+1}' for i in range(vectors_manual.shape[1])])
df_test_vectors = pd.DataFrame(user_vecs, columns=[f'lf_{i+1}' for i in range(user_vecs.shape[1])])
combined_df= pd.concat([df_vectors_manual, df_test_vectors, gts], axis=1)


In [17]:
amazon_treerules_for_user_input = gen_amazon_funcs()

In [18]:
user_input_df

,old_text,expected_label,fold,cid,text,model_pred_diff,model_pred_prob_tuple,model_pred,vectors
1934,would not recommend to anyone the authors choppy language and phrasing was not effective for me the long sequences did not serve a purpose for me the ending in particular left me hanging,1,0.0,165830,would not recommend to anyone the authors choppy language and phrasing was not effective for me the long sequences did not serve a purpose for me the ending in particular left me hanging,0.346848,"(0.3265761919141184, 0.6734238080858815)",1,"-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1"
4035,the curtains themselves are a mesh fishnet which i prefer and i do like them this review is concerning the misleading title it says this purchase includes two pair the seller only sent one pair\nthe curtains themselves are a mesh fishnet which i prefer and i do like them,1,1.0,196572,the curtains themselves are a mesh fishnet which i prefer and i do like them this review is concerning the misleading title it says this purchase includes two pair the seller only sent one pair\nthe curtains themselves are a mesh fishnet which i prefer and i do like them,0.346848,"(0.3265761919141184, 0.6734238080858815)",1,"-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1"
11862,terrible product quality that should be avoided terrible quality and priced too high this is probably the only product ive purchased from amazon that i was outright disgusted with for the price you pay you expect the mug to actually have higher quality decals lettering but no its cheaply painted on and can be scraped away with a thumbnail mine came in with half the lettering on one side rubbed off and running as if it has been hit with hot water and soap the box was also open\n\ndo not buy this product,1,0.0,176497,terrible product quality that should be avoided terrible quality and priced too high this is probably the only product ive purchased from amazon that i was outright disgusted with for the price you pay you expect the mug to actually have higher quality decals lettering but no its cheaply painted on and can be scraped away with a thumbnail mine came in with half the lettering on one side rubbed off and running as if it has been hit with hot water and soap the box was also open\n\ndo not buy this product,0.999584,"(0.00020808575500634775, 0.9997919142449936)",1,"-1,1,-1,-1,-1,0,1,-1,-1,-1,-1,1,-1,1,1"
4073,ordered one defective flashes and clicks once per ordered one defective flashes and clicks once per second returned and replacement would not hold zero mounts were tight a plus is that it is lighter than older models but if it doesnt work it is junk,1,0.0,135568,ordered one defective flashes and clicks once per ordered one defective flashes and clicks once per second returned and replacement would not hold zero mounts were tight a plus is that it is lighter than older models but if it doesnt work it is junk,0.999999,"(3.891488923288918e-07, 0.9999996108511077)",1,"-1,-1,-1,-1,-1,-1,1,1,-1,-1,-1,1,-1,1,-1"
360,cheap quality cheap and the material it peeling from the sole after 1 wearing will use in the back yard,1,0.0,177722,cheap quality cheap and the material it peeling from the sole after 1 wearing will use in the back yard,0.701975,"(0.14901237239774964, 0.8509876276022504)",1,"-1,1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1"
10212,five stars excellent,0,0.0,40914,five stars excellent,0.999978,"(0.9999891506271412, 1.0849372858688671e-05)",0,"0,-1,-1,-1,-1,-1,-1,-1,0,0,-1,-1,-1,-1,-1"
3016,five stars my granddaughter loved this good christmas gift,0,1.0,86878,five stars my granddaughter loved this good christmas gift,0.999978,"(0.9999891506271412, 1.0849372858688671e-05)",0,"0,-1,-1,-1,-1,-1,-1,-1,0,0,-1,-1,-1,-1,-1"
2483,you will love this series if you were a buffy fan you will love this series you will find yourself breathing a little faster as the author does a great job of keeping you on the edge of your seat she also has a gift for making the characters come to life right before y

1. add an instance abstain_threshold, i.e, cant have over abstain_threshold abstains for each instance.
2. instance_accuracy = |accurate preds| / |non-abstain preds|
3. labelling_function_accuracy = |accurate preds| / |non-abstain preds|

In [19]:
# def lf_constraint_solve(df, lf_acc_thresh=0.5, 
#                         instance_acc_thresh=0.5, 
#                         nlf_prefix='nlf_',
#                         abstain_threshold=0.5,
#                         expected_label_col='expected_label',
#                         new_lf_weight=1):
    
#     # Problem initialization
#     prob = pulp.LpProblem("Label_Flip_Minimization", pulp.LpMinimize)

#     # Parameters
# #     labeling_functions = df.columns[:-1]
#     labeling_functions = [lf_name for lf_name in df.columns if lf_name!=expected_label_col]
#     print(f"lf_acc: {lf_acc_thresh}, ins_acc:{instance_acc_thresh}")
#     print(f"labeling_functions: {labeling_functions}")
#     num_instances = len(df)
#     print(f"num_instances: {num_instances}")
#     M = 1000 

#     nlfs = [lf for lf in labeling_functions if nlf_prefix in lf]
#     print(f"nlfs: {nlfs}")
#     x_nlfs = pulp.LpVariable.dicts("x_nlf", nlfs, cat='Binary')

#     P_vars = pulp.LpVariable.dicts("P", (range(num_instances), labeling_functions), 
#                                    lowBound=-1, upBound=1, cat='Integer')
    
#     is_abstain = pulp.LpVariable.dicts("is_abstain", 
#                                    (range(num_instances), labeling_functions), 
#                                    cat='Binary')

#     flip_1_to_0 = pulp.LpVariable.dicts("flip_1_to_0", 
#                                         (range(num_instances), labeling_functions), cat='Binary')
#     flip_1_to_neg1 = pulp.LpVariable.dicts("flip_1_to_neg1", 
#                                            (range(num_instances), labeling_functions), cat='Binary')
#     flip_0_to_1 = pulp.LpVariable.dicts("flip_0_to_1", 
#                                         (range(num_instances), labeling_functions), cat='Binary')
#     flip_0_to_neg1 = pulp.LpVariable.dicts("flip_0_to_neg1", 
#                                            (range(num_instances), labeling_functions), cat='Binary')
#     flip_neg1_to_1 = pulp.LpVariable.dicts("flip_neg1_to_1", 
#                                            (range(num_instances), labeling_functions), cat='Binary')
#     flip_neg1_to_0 = pulp.LpVariable.dicts("flip_neg1_to_0", 
#                                            (range(num_instances), labeling_functions), cat='Binary')

#     # Binary variables to track correctness of predictions (1 if correct, 0 if not)
#     correctness_vars = pulp.LpVariable.dicts("correct", 
#                                              (range(num_instances), labeling_functions), cat='Binary')

#     # Objective: Minimize the number of flips
#     flip_cost = pulp.lpSum([flip_1_to_0[i][lf] + flip_1_to_neg1[i][lf] + 
#                             flip_0_to_1[i][lf] + flip_0_to_neg1[i][lf] + 
#                             flip_neg1_to_1[i][lf] + flip_neg1_to_0[i][lf] 
#                             for i in range(num_instances) for lf in labeling_functions])

#     # prob += flip_cost + new_lf_weight*(x_nlf1+x_nlf2), "Minimize_Flips"
#     # maybe we allow deletion of the exising functions as well!
#     prob += flip_cost + pulp.lpSum([new_lf_weight * x_nlfs[lf] for lf in nlfs]), "Minimize_Flips"


#     # Mutual exclusivity
#     for i in range(num_instances):
#         for lf in labeling_functions:
#             prob += (flip_1_to_0[i][lf] + flip_1_to_neg1[i][lf] + 
#                      flip_0_to_1[i][lf] + flip_0_to_neg1[i][lf] + 
#                      flip_neg1_to_1[i][lf] + flip_neg1_to_0[i][lf]) <= 1, f"Flip_Exclusivity_{i}_{lf}"

#     for i in range(num_instances):
#         for lf in labeling_functions:
#             original_val = df.loc[i, lf]
#             if original_val == 1:
#                 prob += P_vars[i][lf] == 0 * flip_1_to_0[i][lf] + \
#                 (-1) * flip_1_to_neg1[i][lf] + 1 * (1 - flip_1_to_0[i][lf] - flip_1_to_neg1[i][lf]), f"Flip_From_1_{i}_{lf}"
                
#             elif original_val == 0:                
#                 prob += P_vars[i][lf] == 1 * flip_0_to_1[i][lf] + \
#                 (-1) * flip_0_to_neg1[i][lf] + 0 * (1 - flip_0_to_1[i][lf] - flip_0_to_neg1[i][lf]), f"Flip_From_0_{i}_{lf}"
                
#             elif original_val == -1:
#                 prob += P_vars[i][lf] == 1 * flip_neg1_to_1[i][lf] + 0 * flip_neg1_to_0[i][lf] + (-1) * (1 - flip_neg1_to_1[i][lf] - flip_neg1_to_0[i][lf]), f"Flip_From_neg1_{i}_{lf}"

                    
#     for i in range(num_instances):
#         for lf in labeling_functions:
#             # Enforce the is_abstain variable to be 1 if P_vars[i][lf] == -1
#             prob += P_vars[i][lf] + (1 - is_abstain[i][lf]) * M >= 0, f"Abstain_UpperBound_{i}_{lf}"
#             prob += P_vars[i][lf] + (1 - is_abstain[i][lf]) * M <= 0, f"Abstain_LowerBound_{i}_{lf}"

                
# #     for lf in labeling_functions:
# #         num_instances_abstain = pulp.lpSum([is_abstain[i][lf] for i in range(num_instances)])
# #         if lf in nlfs:
# #             lf_correct_predictions = pulp.lpSum([correctness_vars[i][lf] for i in range(num_instances)])
# #             # Apply Big-M method for conditional inclusion of nLFs
# #             prob += lf_correct_predictions >= lf_acc_thresh * (num_instances-num_instances_abstain) - M * (1 - x_nlfs[lf]), f"LF_{lf}_Accuracy"
# #         else:
# #             lf_correct_predictions = pulp.lpSum([correctness_vars[i][lf] for i in range(num_instances)])
# #             prob += lf_correct_predictions >= lf_acc_thresh * (num_instances-num_instances_abstain), f"LF_{lf}_Accuracy"

#     for lf in labeling_functions:
#         if lf in nlfs:
#             lf_correct_predictions = pulp.lpSum([correctness_vars[i][lf] for i in range(num_instances)])
#             # Apply Big-M method for conditional inclusion of nLFs
#             prob += lf_correct_predictions >= lf_acc_thresh * num_instances - M * (1 - x_nlfs[lf]), f"LF_{lf}_Accuracy"
#         else:
#             lf_correct_predictions = pulp.lpSum([correctness_vars[i][lf] for i in range(num_instances)])
#             prob += lf_correct_predictions >= lf_acc_thresh * num_instances, f"LF_{lf}_Accuracy"

            
#     for i in range(num_instances):
#         for lf in nlfs:
#             # Ensure that correctness_vars[i][lf] is counted only if x_nlf[lf] = 1
#             prob += correctness_vars[i][lf] <= M * x_nlfs[lf], f"{lf}_active_{i}"
#         correct_predictions_per_instance = pulp.lpSum([correctness_vars[i][lf] for lf in labeling_functions if lf not in nlfs]) + pulp.lpSum([correctness_vars[i][lf] for lf in nlfs])
#         num_labeling_functions_used = len(labeling_functions) - len(nlfs) + pulp.lpSum(x_nlfs.values())  # Adjust based on active nLFs
#         prob += correct_predictions_per_instance >= instance_acc_thresh * num_labeling_functions_used, f"Instance_{i}_Accuracy"

# #     for i in range(num_instances):
# #         for lf in nlfs:
# #             prob += correctness_vars[i][lf] <= M * x_nlfs[lf], f"{lf}_active_{i}"
# #         # Count the number of non-abstain predictions for each instance
# #         instance_abstain_count = pulp.lpSum([is_abstain[i][lf] for lf in labeling_functions])
        
# #         correct_predictions_per_instance = pulp.lpSum([correctness_vars[i][lf] for lf in labeling_functions if lf not in nlfs]) + pulp.lpSum([correctness_vars[i][lf] for lf in nlfs])
# #         num_labeling_functions_used = len(labeling_functions) - len(nlfs) + pulp.lpSum(x_nlfs.values())-instance_abstain_count  # Adjust based on active nLFs

# #         # Ensure the correct predictions exceed the threshold for non-abstaining functions
# #         prob += correct_predictions_per_instance >= instance_acc_thresh * num_labeling_functions_used, f"Instance_{i}_Accuracy"
# #         prob += instance_abstain_count <= num_labeling_functions_used * abstain_threshold, f"Instance_{i}_non_abstain_req"

#     # Ensure correctness tracking between P_vars and true labels
#     for i in range(num_instances):
#         for lf in labeling_functions:
#             true_label = df[expected_label_col][i]
#             # Ensure that correctness_vars[i][lf] is 1 if P_vars[i][lf] equals true_label, else 0
#             prob += P_vars[i][lf] - true_label <= M * (1 - correctness_vars[i][lf]),\
#                                      f"Correctness_UpperBound_{i}_{lf}"
#             prob += true_label - P_vars[i][lf] <= M * (1 - correctness_vars[i][lf]), \
#                                      f"Correctness_LowerBound_{i}_{lf}"

#     prob.solve()

#     p_vars_solution = pd.DataFrame(index=df.index, columns=labeling_functions)

#     for i in range(num_instances):
#         for lf in labeling_functions:
#             p_vars_solution.loc[i, lf] = int(pulp.value(P_vars[i][lf]))
    
#     correctness_solution = pd.DataFrame(index=df.index, columns=labeling_functions)
#     for i in range(num_instances):
#         for lf in labeling_functions:
#             correctness_solution.loc[i, lf] = int(pulp.value(correctness_vars[i][lf]))
    
#     x_nlfs_solution = {lf: pulp.value(x_nlfs[lf]) for lf in nlfs}
    
#     print(f"Status: {pulp.LpStatus[prob.status]}")
    
#     return p_vars_solution, x_nlfs_solution, prob

In [20]:
def lf_constraint_solve(df, lf_acc_thresh=0.5, 
                        instance_acc_thresh=0.5,
                        min_non_abstain_thresh=0.8,
                        nlf_prefix='nlf_',
                        expected_label_col='expected_label',
                        new_lf_weight=1):
    
    # Problem initialization
    prob = pulp.LpProblem("Label_Flip_Minimization", pulp.LpMinimize)

    # Parameters
#     labeling_functions = df.columns[:-1]
    labeling_functions = [lf_name for lf_name in df.columns if lf_name!=expected_label_col]
    print(f"lf_acc: {lf_acc_thresh}, ins_acc:{instance_acc_thresh}")
    print(f"labeling_functions: {labeling_functions}")
    num_instances = len(df)
    print(f"num_instances: {num_instances}")
    M = 5
    
    nlfs = [lf for lf in labeling_functions if nlf_prefix in lf]
    print(f"nlfs: {nlfs}")
    x_nlfs = pulp.LpVariable.dicts("x_nlf", nlfs, cat='Binary')

    P_vars = pulp.LpVariable.dicts("P", (range(num_instances), labeling_functions), 
                                   lowBound=-1, upBound=1, cat='Integer')
    
    is_abstain = pulp.LpVariable.dicts("is_abstain", 
                               (range(num_instances), labeling_functions), 
                               cat='Binary')

    flip_1_to_0 = pulp.LpVariable.dicts("flip_1_to_0", 
                                        (range(num_instances), labeling_functions), cat='Binary')
    flip_1_to_neg1 = pulp.LpVariable.dicts("flip_1_to_neg1", 
                                           (range(num_instances), labeling_functions), cat='Binary')
    flip_0_to_1 = pulp.LpVariable.dicts("flip_0_to_1", 
                                        (range(num_instances), labeling_functions), cat='Binary')
    flip_0_to_neg1 = pulp.LpVariable.dicts("flip_0_to_neg1", 
                                           (range(num_instances), labeling_functions), cat='Binary')
    flip_neg1_to_1 = pulp.LpVariable.dicts("flip_neg1_to_1", 
                                           (range(num_instances), labeling_functions), cat='Binary')
    flip_neg1_to_0 = pulp.LpVariable.dicts("flip_neg1_to_0", 
                                           (range(num_instances), labeling_functions), cat='Binary')

    # Binary variables to track correctness of predictions (1 if correct, 0 if not)
    correctness_vars = pulp.LpVariable.dicts("correct", 
                                             (range(num_instances), labeling_functions), cat='Binary')
    
    # Create auxiliary variables to represent active nLF abstains
    active_abstain = pulp.LpVariable.dicts("active_abstain", 
                                           (range(num_instances), nlfs), 
                                           cat='Binary')
    
    correct_and_active = pulp.LpVariable.dicts("correct_and_active", 
                                           (range(num_instances), nlfs), 
                                           cat='Binary')


    # Objective: Minimize the number of flips
    flip_cost = pulp.lpSum([flip_1_to_0[i][lf] + flip_1_to_neg1[i][lf] + 
                            flip_0_to_1[i][lf] + flip_0_to_neg1[i][lf] + 
                            flip_neg1_to_1[i][lf] + flip_neg1_to_0[i][lf] 
                            for i in range(num_instances) for lf in labeling_functions])

    prob += flip_cost + pulp.lpSum([new_lf_weight * x_nlfs[lf] for lf in nlfs]), "Minimize_Flips"


    # Mutual exclusivity
    for i in range(num_instances):
        for lf in labeling_functions:
            prob += (flip_1_to_0[i][lf] + flip_1_to_neg1[i][lf] + 
                     flip_0_to_1[i][lf] + flip_0_to_neg1[i][lf] + 
                     flip_neg1_to_1[i][lf] + flip_neg1_to_0[i][lf]) <= 1, f"Flip_Exclusivity_{i}_{lf}"

    for i in range(num_instances):
        for lf in labeling_functions:
            original_val = df.loc[i, lf]
            if original_val == 1:
                prob += P_vars[i][lf] == 0 * flip_1_to_0[i][lf] + \
                (-1) * flip_1_to_neg1[i][lf] + 1 * (1 - flip_1_to_0[i][lf] - flip_1_to_neg1[i][lf]), f"Flip_From_1_{i}_{lf}"
                
            elif original_val == 0:                
                prob += P_vars[i][lf] == 1 * flip_0_to_1[i][lf] + \
                (-1) * flip_0_to_neg1[i][lf] + 0 * (1 - flip_0_to_1[i][lf] - flip_0_to_neg1[i][lf]), f"Flip_From_0_{i}_{lf}"
                
            elif original_val == -1:
                prob += P_vars[i][lf] == 1 * flip_neg1_to_1[i][lf] + 0 * flip_neg1_to_0[i][lf] + (-1) * (1 - flip_neg1_to_1[i][lf] - flip_neg1_to_0[i][lf]), f"Flip_From_neg1_{i}_{lf}"
    
    for i in range(num_instances):
        for lf in labeling_functions:
            prob += P_vars[i][lf] >= -1 - (1 - is_abstain[i][lf]) * M, f"Abstain_LowerBound_{i}_{lf}"
            prob += P_vars[i][lf] <= -1 + (1 - is_abstain[i][lf]) * M, f"Abstain_UpperBound_{i}_{lf}"

            # If is_abstain[i][lf] == 0, P_vars[i][lf] can only be 0 or 1
            prob += P_vars[i][lf] >= 0 - is_abstain[i][lf] * M, f"Non_Abstain_LowerBound_{i}_{lf}"
            prob += P_vars[i][lf] <= 1 + is_abstain[i][lf] * M, f"Non_Abstain_UpperBound_{i}_{lf}"
    
    # Set up the constraints for the auxiliary variables
    for i in range(num_instances):
        for lf in nlfs:
            # Ensure active_abstain[i][lf] is 1 only if both is_abstain[i][lf] == 1 and x_nlfs[lf] == 1
            prob += active_abstain[i][lf] <= is_abstain[i][lf], f"ActiveAbstain_LF_{lf}_Instance_{i}_1"
            prob += active_abstain[i][lf] <= x_nlfs[lf], f"ActiveAbstain_LF_{lf}_Instance_{i}_2"
            prob += active_abstain[i][lf] >= is_abstain[i][lf] + x_nlfs[lf] - 1, f"ActiveAbstain_LF_{lf}_Instance_{i}_3"

    for i in range(num_instances):
        for lf in nlfs:
            # correct_and_active[i][lf] should be 1 only if both correctness_vars[i][lf] == 1 and x_nlfs[lf] == 1
            prob += correct_and_active[i][lf] <= correctness_vars[i][lf], f"CorrectAndActive_UpperBound_1_{i}_{lf}"
            prob += correct_and_active[i][lf] <= x_nlfs[lf], f"CorrectAndActive_UpperBound_2_{i}_{lf}"
            prob += correct_and_active[i][lf] >= correctness_vars[i][lf] + x_nlfs[lf] - 1, f"CorrectAndActive_LowerBound_{i}_{lf}"
        
    
    for lf in labeling_functions:
        num_instances_abstain = pulp.lpSum([is_abstain[i][lf] for i in range(num_instances)])
        if lf in nlfs:
            lf_correct_predictions = pulp.lpSum([correctness_vars[i][lf] for i in range(num_instances)])
            prob += lf_correct_predictions >= lf_acc_thresh * (num_instances-num_instances_abstain) - M * (1 - x_nlfs[lf]), f"LF_{lf}_Accuracy"
        else:
            lf_correct_predictions = pulp.lpSum([correctness_vars[i][lf] for i in range(num_instances)])
            prob += lf_correct_predictions >= lf_acc_thresh * (num_instances-num_instances_abstain), f"LF_{lf}_Accuracy"



    for i in range(num_instances):
        for lf in nlfs:
            # Ensure that correctness_vars[i][lf] is counted only if x_nlf[lf] = 1
            prob += correctness_vars[i][lf] <= M * x_nlfs[lf], f"{lf}_active_{i}"
            
        correct_predictions_per_instance = pulp.lpSum([correctness_vars[i][lf] for lf in labeling_functions if lf not in nlfs]) + \
                               pulp.lpSum([correct_and_active[i][lf] for lf in nlfs])
        instance_abstain_count = pulp.lpSum([is_abstain[i][lf] for lf in labeling_functions if lf not in nlfs]) + \
                                 pulp.lpSum([active_abstain[i][lf] for lf in nlfs]) 
        
        num_labeling_functions_used = len(labeling_functions) - len(nlfs) + pulp.lpSum(x_nlfs.values())
        prob += correct_predictions_per_instance >= instance_acc_thresh * num_labeling_functions_used, f"Instance_{i}_Accuracy"
        prob += instance_abstain_count <= num_labeling_functions_used *(1- min_non_abstain_thresh), f"Instance_{i}_NonAbastain"

        
    for i in range(num_instances):
        for lf in labeling_functions:
            true_label = df[expected_label_col][i]
            # Ensure that correctness_vars[i][lf] is 1 if P_vars[i][lf] equals true_label, else 0
            prob += P_vars[i][lf] - true_label <= M * (1 - correctness_vars[i][lf]),\
                                     f"Correctness_UpperBound_{i}_{lf}"
            prob += true_label - P_vars[i][lf] <= M * (1 - correctness_vars[i][lf]), \
                                     f"Correctness_LowerBound_{i}_{lf}"


    # Solve the integer program
    prob.solve()

    p_vars_solution = pd.DataFrame(index=df.index, columns=labeling_functions)
    active_abstain_df = pd.DataFrame(index=df.index, columns=labeling_functions)
    is_abstain_df = pd.DataFrame(index=df.index, columns=labeling_functions)
    
    for i in range(num_instances):
        for lf in labeling_functions:
            p_vars_solution.loc[i, lf] = int(pulp.value(P_vars[i][lf]))
    
    correctness_solution = pd.DataFrame(index=df.index, columns=labeling_functions)
    for i in range(num_instances):
        for lf in labeling_functions:
            correctness_solution.loc[i, lf] = int(pulp.value(correctness_vars[i][lf]))
    
    x_nlfs_solution = {lf: pulp.value(x_nlfs[lf]) for lf in nlfs}
    
    print(f"Status: {pulp.LpStatus[prob.status]}")
    print(f"pulp.value(num_labeling_functions_used) : {pulp.value(num_labeling_functions_used)}")
    
    for i in range(num_instances):
        for lf in labeling_functions:
            is_abstain_df.loc[i, lf] = int(pulp.value(is_abstain[i][lf]))
    for i in range(num_instances):
        for lf in nlfs:
            active_abstain_df.loc[i, lf] = int(pulp.value(active_abstain[i][lf]))
    
    return p_vars_solution, x_nlfs_solution, pulp, prob, active_abstain_df, is_abstain_df


In [21]:
# user_fixed_input_matrix = res_df[lf_names_after_fix].values.astype(int)

In [22]:
# true_labels_vec = gts.values.astype(int)

In [23]:
# label_model = LabelModel(cardinality=2, verbose=True)

In [24]:
# user_fixed_input_matrix

In [25]:
# label_model.fit(L_train=user_fixed_input_matrix, n_epochs=100, log_freq=10, seed=42)


In [26]:
# predicted_labels_new = label_model.predict(L=user_fixed_input_matrix)

In [27]:
list(combined_df)

['nlf_1',
 'nlf_2',
 'nlf_3',
 'nlf_4',
 'nlf_5',
 'nlf_6',
 'nlf_7',
 'nlf_8',
 'nlf_9',
 'nlf_10',
 'nlf_11',
 'nlf_12',
 'nlf_13',
 'lf_1',
 'lf_2',
 'lf_3',
 'lf_4',
 'lf_5',
 'lf_6',
 'lf_7',
 'lf_8',
 'lf_9',
 'lf_10',
 'lf_11',
 'lf_12',
 'lf_13',
 'lf_14',
 'lf_15',
 'expected_label']

In [28]:
combined_df

,nlf_1,nlf_2,nlf_3,nlf_4,nlf_5,nlf_6,nlf_7,nlf_8,nlf_9,nlf_10,...,lf_7,lf_8,lf_9,lf_10,lf_11,lf_12,lf_13,lf_14,lf_15,expected_label
0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,-1,1
1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,-1,1
2,-1,-1,-1,-1,-1,-1,1,-1,1,1,...,1,-1,-1,-1,-1,1,-1,1,1,1
3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,1,-1,-1,-1,1,-1,1,-1,1
4,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,-1,1
5,-1,0,-1,0,-1,-1,-1,-1,-1,-1,...,-1,-1,0,0,-1,-1,-1,-1,-1,0
6,0,0,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,0,0,-1,-1,-1,-1,-1,0
7,0,0,-1,-1,-1,-1,-1,-1,-1,-1,...,1,-1,0,-1,-1,-1,-1,-1,-1,0
8,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,-1,1
9,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,1,-1,-1,-1,1,-1,1,-1,1


In [29]:
for c in list(combined_df):
    print(f"{c}: {combined_df[c].value_counts().to_dict()}")

nlf_1: {-1: 16, 0: 4}
nlf_2: {-1: 16, 0: 4}
nlf_3: {-1: 20}
nlf_4: {-1: 19, 0: 1}
nlf_5: {-1: 19, 0: 1}
nlf_6: {-1: 19, 0: 1}
nlf_7: {-1: 17, 1: 3}
nlf_8: {-1: 20}
nlf_9: {-1: 15, 1: 5}
nlf_10: {-1: 19, 1: 1}
nlf_11: {-1: 20}
nlf_12: {-1: 12, 1: 8}
nlf_13: {-1: 20}
lf_1: {-1: 14, 0: 6}
lf_2: {-1: 14, 1: 6}
lf_3: {-1: 18, 0: 2}
lf_4: {-1: 20}
lf_5: {-1: 19, 1: 1}
lf_6: {-1: 15, 0: 5}
lf_7: {1: 15, -1: 5}
lf_8: {-1: 17, 1: 3}
lf_9: {-1: 13, 0: 7}
lf_10: {-1: 18, 0: 2}
lf_11: {-1: 18, 0: 2}
lf_12: {-1: 15, 1: 5}
lf_13: {-1: 20}
lf_14: {-1: 17, 1: 3}
lf_15: {-1: 19, 1: 1}
expected_label: {1: 10, 0: 10}


In [30]:
def evaluate_snorkel_acc(true_labels, pred_labels):
#     true_labels = original_predictions['tlabel'].values

    # Accuracy
    accuracy = accuracy_score(true_labels, pred_labels)
    print(f"Accuracy: {accuracy:.4f}")

    # Detailed classification report (precision, recall, F1-score)
    print(classification_report(true_labels, pred_labels))

In [31]:
# true_labels_vec.shape

In [32]:
# true_labels_vec

In [33]:
# predicted_labels_new.shape

In [34]:
# evaluate_snorkel_acc()
# evaluate_snorkel_acc(true_labels=true_labels_vec, pred_labels=predicted_labels_new)

In [35]:
# res_df[lf_names_after_fix]

In [36]:
# res_df[lf_names_after_fix].head()

In [37]:
# list(user_input_df)

In [38]:
# user_input_df

In [39]:
def create_solver_input_df_copies(lf_names_after_fix, user_input_df, res_df):
    df_copies = {}

    cols_needed = ['text', 'expected_label', 'cid']

    # Loop through each column in df2 and create a copy of df1 with modified 'expected_label'
    for lf in lf_names_after_fix:
        # Create a deep copy of df1
        df_copy = user_input_df.copy(deep=True)

        # Update the 'expected_label' column based on the corresponding column in df2
        df_copy['expected_label'] = res_df[lf].values

        # Store the modified dataframe in the dictionary with key as the labeling function name
        df_copies[lf] = df_copy[cols_needed]
    
    return df_copies



In [40]:
lfs_manual = [l for l in list(combined_df) if 'nlf' in l]

In [41]:
lfs_manual

['nlf_1',
 'nlf_2',
 'nlf_3',
 'nlf_4',
 'nlf_5',
 'nlf_6',
 'nlf_7',
 'nlf_8',
 'nlf_9',
 'nlf_10',
 'nlf_11',
 'nlf_12',
 'nlf_13']

In [42]:
manual_func_dict = dict(zip(lfs_manual,manual_tree_rules))

In [43]:
# manual_func_dict['nlf_dummy']=None

In [44]:
combined_df

,nlf_1,nlf_2,nlf_3,nlf_4,nlf_5,nlf_6,nlf_7,nlf_8,nlf_9,nlf_10,...,lf_7,lf_8,lf_9,lf_10,lf_11,lf_12,lf_13,lf_14,lf_15,expected_label
0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,-1,1
1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,-1,1
2,-1,-1,-1,-1,-1,-1,1,-1,1,1,...,1,-1,-1,-1,-1,1,-1,1,1,1
3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,1,-1,-1,-1,1,-1,1,-1,1
4,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,-1,1
5,-1,0,-1,0,-1,-1,-1,-1,-1,-1,...,-1,-1,0,0,-1,-1,-1,-1,-1,0
6,0,0,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,0,0,-1,-1,-1,-1,-1,0
7,0,0,-1,-1,-1,-1,-1,-1,-1,-1,...,1,-1,0,-1,-1,-1,-1,-1,-1,0
8,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,-1,1
9,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,1,-1,-1,-1,1,-1,1,-1,1


In [55]:
import numpy as np

test_res = {}

res_df, inclusion_dict, res_pulp, prob, active_abstain_df, is_abstain_df = lf_constraint_solve(df=combined_df, lf_acc_thresh=1,
                instance_acc_thresh=0.8,
                min_non_abstain_thresh=0.5,      
                nlf_prefix='nlf_',
                expected_label_col='expected_label',
                new_lf_weight=1)


#         inclusion_dict['nlf_3']=1
#         # testing purpose, remember to delete

lfs_witan = [l for l in list(combined_df) if ('nlf' not in l and l!='expected_label')]
lfs_manual_added =  [x for x in inclusion_dict if inclusion_dict[x]==1]
lf_names_after_fix = lfs_witan +lfs_manual_added

# print(f"lf_names_after_fix: {lf_names_after_fix}")
# df_copies = create_solver_input_df_copies(lf_names_after_fix=lf_names_after_fix,
#                                  user_input_df=user_input_df,
#                                  res_df=res_df)

# fix_book_keeping_dict = {k.id:{'rule':k, 'deleted':False,
#                        'pre_fix_size':k.size, 
#                        'after_fix_size':k.size, 
#                        'pre-deleted': False} for k in amazon_treerules}

# max_rule_id = max(list(fix_book_keeping_dict))+1

# if(lfs_manual_added):
#     for lma in lfs_manual_added:
#         fix_book_keeping_dict[max_rule_id]={'rule':manual_func_dict[lma], 'deleted':False,
#                        'pre_fix_size':manual_func_dict[lma], 
#                        'after_fix_size':manual_func_dict[lma], 
#                        'pre-deleted': False}
#         max_rule_id+=1


lf_acc: 1, ins_acc:0.8
labeling_functions: ['nlf_1', 'nlf_2', 'nlf_3', 'nlf_4', 'nlf_5', 'nlf_6', 'nlf_7', 'nlf_8', 'nlf_9', 'nlf_10', 'nlf_11', 'nlf_12', 'nlf_13', 'lf_1', 'lf_2', 'lf_3', 'lf_4', 'lf_5', 'lf_6', 'lf_7', 'lf_8', 'lf_9', 'lf_10', 'lf_11', 'lf_12', 'lf_13', 'lf_14', 'lf_15']
num_instances: 20
nlfs: ['nlf_1', 'nlf_2', 'nlf_3', 'nlf_4', 'nlf_5', 'nlf_6', 'nlf_7', 'nlf_8', 'nlf_9', 'nlf_10', 'nlf_11', 'nlf_12', 'nlf_13']


DEBUG 2024-10-13 01:25:13,728 [coin_api.py:solve_CBC:165] /home/opc/.pyenv/versions/3.8.0/envs/label/lib/python3.8/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/1df6076b55bf44f0bc72c9cfad6b66ab-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/1df6076b55bf44f0bc72c9cfad6b66ab-pulp.sol 


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/opc/.pyenv/versions/3.8.0/envs/label/lib/python3.8/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/1df6076b55bf44f0bc72c9cfad6b66ab-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/1df6076b55bf44f0bc72c9cfad6b66ab-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 6373 COLUMNS
At line 39586 RHS
At line 45955 BOUNDS
At line 52089 ENDATA
Problem MODEL has 6368 rows, 5573 columns and 18693 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 24 - 0.10 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 1953 strengthened rows, 695 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1020 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 62 strengthened rows, 0 substitutions
Cgl0004I processed model has 3362 rows, 2981 columns (2981 integer (2427 of which b

In [56]:
inclusion_dict

{'nlf_1': 0.0,
 'nlf_2': 0.0,
 'nlf_3': 0.0,
 'nlf_4': 0.0,
 'nlf_5': 0.0,
 'nlf_6': 0.0,
 'nlf_7': 0.0,
 'nlf_8': 0.0,
 'nlf_9': 0.0,
 'nlf_10': 0.0,
 'nlf_11': 0.0,
 'nlf_12': 0.0,
 'nlf_13': 0.0}

In [57]:
res_df

,nlf_1,nlf_2,nlf_3,nlf_4,nlf_5,nlf_6,nlf_7,nlf_8,nlf_9,nlf_10,...,lf_6,lf_7,lf_8,lf_9,lf_10,lf_11,lf_12,lf_13,lf_14,lf_15
0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,1,1,1,1,1,1,1,1
1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,...,1,1,1,-1,1,1,1,1,1,1
2,-1,-1,-1,-1,-1,-1,1,-1,1,1,...,1,1,-1,1,1,1,1,1,1,1
3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,1,1,1,1,1,-1,1,1
4,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,...,-1,1,-1,1,1,1,1,1,1,1
5,-1,0,-1,0,-1,-1,-1,-1,-1,-1,...,0,-1,0,0,0,0,-1,-1,0,0
6,0,0,-1,-1,-1,-1,-1,-1,-1,-1,...,0,-1,0,0,0,-1,-1,0,0,0
7,0,0,-1,-1,-1,-1,-1,-1,-1,-1,...,0,0,0,0,-1,0,0,0,-1,0
8,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,1,1,1,-1,1,1,1,1,-1
9,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,1,1,-1,-1,1,1,1,1,1


In [59]:
res_with_included_lf = res_df[lf_names_after_fix]

In [60]:
res_with_included_lf['gts']=gts

/tmp/ipykernel_3802/3672333084.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_with_included_lf['gts']=gts


In [61]:
res_with_included_lf

,lf_1,lf_2,lf_3,lf_4,lf_5,lf_6,lf_7,lf_8,lf_9,lf_10,lf_11,lf_12,lf_13,lf_14,lf_15,gts
0,-1,1,1,1,-1,-1,1,1,1,1,1,1,1,1,1,1
1,1,-1,1,1,-1,1,1,1,-1,1,1,1,1,1,1,1
2,1,1,-1,-1,1,1,1,-1,1,1,1,1,1,1,1,1
3,1,1,1,1,-1,-1,1,1,1,1,1,1,-1,1,1,1
4,1,1,1,1,-1,-1,1,-1,1,1,1,1,1,1,1,1
5,0,0,0,0,0,0,-1,0,0,0,0,-1,-1,0,0,0
6,0,0,0,0,0,0,-1,0,0,0,-1,-1,0,0,0,0
7,0,0,0,0,-1,0,0,0,0,-1,0,0,0,-1,0,0
8,1,1,1,1,-1,1,1,1,1,-1,1,1,1,1,-1,1
9,1,1,-1,1,1,1,1,1,-1,-1,1,1,1,1,1,1


In [50]:
active_abstain_df[lf_names_after_fix]

,lf_1,lf_2,lf_3,lf_4,lf_5,lf_6,lf_7,lf_8,lf_9,lf_10,lf_11,lf_12,lf_13,lf_14,lf_15,nlf_12
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [52]:
# instance accuracy: |correct_predictions_from_included_lfs|/|included_lfs|
# lf accuracy: |correct_predictions_from_each_lf|/|non_abstain_preds_from_the_lf|
# instance_non_abstain_thresh: each instance cant have more than (instance_non_abstain_thresh*100)% abstains

In [53]:
is_abstain_df[lf_names_after_fix]

,lf_1,lf_2,lf_3,lf_4,lf_5,lf_6,lf_7,lf_8,lf_9,lf_10,lf_11,lf_12,lf_13,lf_14,lf_15,nlf_12
0,0,0,1,0,1,1,0,1,1,0,0,1,0,1,1,0
1,1,1,0,0,1,1,0,0,0,0,1,0,1,1,1,0
2,1,0,1,0,1,0,0,1,1,1,1,0,1,0,0,0
3,1,1,1,1,0,0,0,0,1,1,1,0,1,0,0,0
4,0,0,1,1,0,1,0,1,1,0,0,1,0,1,1,0
5,0,0,0,1,0,1,0,1,0,0,1,1,1,1,1,0
6,0,0,1,1,1,1,0,1,0,0,0,1,0,1,1,0
7,0,0,0,1,1,0,0,1,0,0,1,1,1,1,1,0
8,1,0,1,1,0,0,0,0,1,1,0,1,0,1,1,0
9,1,1,1,1,0,0,0,0,1,1,1,0,0,0,1,0


In [ ]:
# for i in test_res:
#     print(f"{i}: {len(test_res[i])}")
#     for j in test_res[i]:
#         print(f'rule {j}')
#         print(test_res[i][j]['rule'])
#         print('\n')
#     print('----------------------------------------')

In [ ]:
# len(test_res)

In [ ]:
# for i in test_res:
#     with open(i, 'wb') as f:
#         pickle.dump(test_res[i], f)

In [ ]:
# inclusion_dict

In [ ]:
# list(fix_book_keeping_dict)

In [ ]:
# lf_names_after_fix

In [ ]:
# lf_names_after_fix = lf_names+[x for x in inclusion_dict if inclusion_dict[x]==1]
# df_copies = create_solver_input_df_copies(lf_names_after_fix=lf_names_after_fix,
#                                  user_input_df=user_input_df,
#                                  res_df=res_df)
# df_list = list(df_copies.values())
# for i in range(len(df_list)):
#     fix_book_keeping_dict[i]['user_input'] = df_list[i]
# fix_book_keeping_dict

In [ ]:
# len(df_list)

In [ ]:
# inclusion_dict

In [ ]:
# inclusion_dict

In [ ]:
# len(fix_book_keeping_dict)

In [ ]:
# import pickle
# with open('amazon_full_la_0.8_ia_0.9_input.pkl', 'wb') as f:
#     pickle.dump(fix_book_keeping_dict, f)